In [2]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1646555837149_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
# sc.install_pypi_package("pip==22.0.3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

In [4]:
# sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 45.4 MB/s eta 0:00:00

In [8]:
# sc.install_pypi_package("Pillow")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 96.1 MB/s eta 0:00:00

In [28]:
# sc.install_pypi_package("pyarrow")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1646556444782-0
    Can't uninstall 'numpy'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python37-sagemaker-pyspark 1.4.1 requires pyspark==2.3.4, which is not installed.
tensorflow 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.
tensorflow 2.4.1 requires gast==0.3.3, but you have gast 0.5.2 which is incompatible.
tensorflow 2.4.1 requires grpcio~=1.32.0, but you have grpcio 1.41.0 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you ha

In [44]:
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# This is just to take a look at the dataset, this dataframe is not really needed
# since the training data will be obtained from the images themselves

df_masks = spark.read.csv('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/df_masks.csv', header=True)
df_masks.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+----+------+---+--------------------------------------------------------------------------------------------+
|ID |TYPE|USER|GENDER|AGE|name                                                                                        |
+---+----+----+------+---+--------------------------------------------------------------------------------------------+
|1  |1   |1   |MALE  |25 |s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000001_1_000001_MALE_25.jpg  |
|1  |4   |1   |MALE  |25 |s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000001_4_000001_MALE_25.jpg  |
|2  |1   |2   |MALE  |23 |s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000002_1_000002_MALE_23.jpg  |
|2  |2   |2   |MALE  |23 |s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000002_2_000002_MALE_23.jpg  |
|2  |3   |2   |MALE  |23 |s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000002_3_000002_MALE_23.jpg  |
|2  |4   |2   |MALE  |23 |s3://pguevara-

In [50]:
# Modify the wildcards in the below filepath to control which/how many images are loaded
# into the images dataframe

images = (spark.read.format("binaryFile").
          option("pathGlobFilter", "*.jpg").
          load('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/images/000???_?_000???_*.jpg'))

images.limit(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, modificationTime: timestamp, length: bigint, content: binary]

In [51]:
images.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-------+--------------------+
|                path|   modificationTime| length|             content|
+--------------------+-------------------+-------+--------------------+
|s3://pguevara-bdc...|2022-02-25 04:09:51|9751733|[FF D8 FF E1 24 4...|
|s3://pguevara-bdc...|2022-02-23 13:29:55|9705346|[FF D8 FF E1 3F 6...|
|s3://pguevara-bdc...|2022-02-23 23:19:35|9439782|[FF D8 FF E1 35 3...|
|s3://pguevara-bdc...|2022-02-24 05:17:41|9356056|[FF D8 FF E1 41 C...|
|s3://pguevara-bdc...|2022-02-24 02:19:23|9119772|[FF D8 FF E1 41 5...|
+--------------------+-------------------+-------+--------------------+
only showing top 5 rows

In [52]:
images.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2495

In [53]:
images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

In [24]:
model = ResNet50(include_top=False)
model.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
__________________________

In [25]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50 model with top layer removed and broadcasted pretrained weights.
  """
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)

In [31]:
# Pandas UDFs on large records (e.g., very large images) can run into Out Of Memory (OOM) errors.
# If you hit such errors in the cell below, try reducing the Arrow batch size via `maxRecordsPerBatch`.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# We can now run featurization on our entire Spark DataFrame.
# NOTE: This can take a long time (about 10 minutes) since it applies a large model to the full dataset.
features_df = images.repartition(16).select(col("path"), featurize_udf("content").alias("features"))
features_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|                path|            features|
+--------------------+--------------------+
|s3://pguevara-bdc...|[0.0, 0.0, 0.0, 0...|
|s3://pguevara-bdc...|[0.0, 0.0, 0.0, 0...|
|s3://pguevara-bdc...|[0.0, 0.0, 0.0, 0...|
|s3://pguevara-bdc...|[0.0, 0.0, 0.0, 0...|
|s3://pguevara-bdc...|[0.0, 0.0, 0.0, 1...|
+--------------------+--------------------+
only showing top 5 rows

In [54]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql import functions as F

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df = (features_df.
      withColumn('label', F.substring('path', 71, 1).cast('int')).
      withColumn('features', list_to_vector_udf(F.col('features'))))


df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|                path|            features|label|
+--------------------+--------------------+-----+
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    1|
|s3://pguevara-bdc...|[0.0,0.0,0.0,1.81...|    2|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    4|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    1|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    1|
+--------------------+--------------------+-----+
only showing top 5 rows

In [61]:
# This is just to save a copy of the featurized dataset (before train-test splitting)
# in parquet format on the s3 bucket
df.write.mode("overwrite").parquet('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/tmp/dataset')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

In [56]:
df_train, df_test = df.randomSplit([0.75, 0.25])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
from pyspark.ml.classification import RandomForestClassifier

rf_class = RandomForestClassifier(featuresCol='features',
                                  labelCol='label',
                                  maxBins=1000, seed=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
rf_trained = rf_class.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
df_predictions = rf_trained.transform(df_test)
df_predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|                path|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    4|[0.0,3.0,1.0,1.0,...|[0.0,0.15,0.05,0....|       4.0|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    4|[0.0,0.0,0.0,3.0,...|[0.0,0.0,0.0,0.15...|       4.0|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    4|[0.0,0.0,0.0,3.0,...|[0.0,0.0,0.0,0.15...|       4.0|
|s3://pguevara-bdc...|[0.0,0.0,0.0,0.0,...|    3|[0.0,0.0,0.0,14.0...|[0.0,0.0,0.0,0.7,...|       3.0|
+--------------------+--------------------+-----+--------------------+--------------------+----------+

In [60]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(df_predictions.select("prediction", "label"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test set accuracy = 1.0